# Introduction
In this tutorial, we will explore how to use various bagging algorithms to predict income levels using the Adult dataset. The Adult dataset, also known as the "Census Income" dataset, contains data on individuals from the 1994 Census database. The task is to predict whether a person's income exceeds $50,000/year based on their demographic attributes.

# Steps


## Step 1: Import Required Libraries
First, import the necessary libraries for data manipulation, model training, and evaluation.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Step 2: Load and Preprocess the Dataset
Load the Adult dataset and preprocess it. This includes handling missing values, encoding categorical variables, and splitting the data into features and target variables.

In [2]:
# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
           'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
data = pd.read_csv(url, header=None, names=columns, na_values=' ?')

# Handle missing values
data = data.dropna()

# Encode categorical variables
data = pd.get_dummies(data, drop_first=True)

# Split the data into features and target variable
X = data.drop('income_ >50K', axis=1)
y = data['income_ >50K']


## Step 3: Split the Dataset
Split the dataset into training and testing sets to evaluate the performance of the models.


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


## Step 4: Initialize and Train the Classifiers



### Bagging Meta-estimator
Initialize a K-Nearest Neighbors classifier and use it as the base estimator for the Bagging classifier.

In [4]:
# Initialize base classifier and Bagging Meta-estimator
base_estimator = KNeighborsClassifier()
bagging_classifier = BaggingClassifier(base_estimator, n_estimators=50, random_state=42)

# Train the classifier on the training data
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test data
predictions = bagging_classifier.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Bagging Classifier Model Accuracy: {accuracy * 100:.2f}%')


Bagging Classifier Model Accuracy: 76.84%


### Random Forest
Initialize and train a Random Forest classifier.

In [ ]:
# Initialize and train the Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
random_forest_classifier.fit(X_train, y_train)

# Make predictions on the test data
predictions = random_forest_classifier.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Random Forest Model Accuracy: {accuracy * 100:.2f}%')


### Pasting
Initialize a Decision Tree classifier and use it as the base estimator for a Bagging classifier with Pasting (without replacement).

In [ ]:
# Initialize the base classifier and Bagging Meta-estimator with Pasting
base_estimator = DecisionTreeClassifier()
pasting_classifier = BaggingClassifier(base_estimator, n_estimators=50, max_samples=0.7, bootstrap=False, random_state=42)

# Train the classifier on the training data
pasting_classifier.fit(X_train, y_train)

# Make predictions on the test data
predictions = pasting_classifier.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Pasting Classifier Model Accuracy: {accuracy * 100:.2f}%')


### Roughly Balanced Bagging (RBB)
Implement Roughly Balanced Bagging by manually creating balanced bootstrap samples and aggregating predictions from multiple Decision Tree classifiers.

In [ ]:
import numpy as np

# Number of base estimators
n_estimators = 100

# Initialize arrays to store the ensemble predictions and models
ensemble_preds = np.zeros((n_estimators, len(X_test)))
ensemble_models = []

for i in range(n_estimators):
    # Create a bootstrap sample, ensuring it's roughly balanced
    pos_indices = np.where(y_train == 1)[0]
    neg_indices = np.where(y_train == 0)[0]

    chosen_pos_indices = np.random.choice(pos_indices, size=len(pos_indices), replace=True)
    chosen_neg_indices = np.random.choice(neg_indices, size=len(pos_indices), replace=True)

    balanced_sample_indices = np.concatenate([chosen_pos_indices, chosen_neg_indices])
    np.random.shuffle(balanced_sample_indices)

    X_train_balanced = X_train.iloc[balanced_sample_indices]
    y_train_balanced = y_train.iloc[balanced_sample_indices]

    # Train a decision tree classifier on the balanced bootstrap sample
    tree_clf = DecisionTreeClassifier(random_state=i)
    tree_clf.fit(X_train_balanced, y_train_balanced)
    ensemble_models.append(tree_clf)

    # Make predictions on the test set
    ensemble_preds[i] = tree_clf.predict(X_test)

# Majority voting across all estimators for the final prediction
final_preds = np.round(np.mean(ensemble_preds, axis=0))

# Calculate accuracy
accuracy = accuracy_score(y_test, final_preds)
print(f'Roughly Balanced Bagging Model Accuracy: {accuracy:.2f}')
